In [ ]:
import streamlit as st
from PIL import Image
from pyzbar.pyzbar import decode
import json

st.set_page_config(page_title="QR Scanner", page_icon="📷", layout="centered")

# --- Function to Decode Image ---
def decode_img(img_path):
    img = Image.open(img_path)
    result = decode(img)
    if result:
        data = result[0].data.decode("utf-8")
        try:
            # Try to parse as JSON (if QR contains JSON)
            return json.loads(data)
        except:
            return {"Decoded Data": data}
    return None

# --- Page Navigation using session_state ---
if "page" not in st.session_state:
    st.session_state.page = "scanner"

def go_to_result(decoded_data):
    st.session_state.page = "result"
    st.session_state.decoded_data = decoded_data

# --- Page 1: Scanner Page ---
if st.session_state.page == "scanner":
    st.title("📷 QR Code Scanner")

    img_file = st.camera_input("Click below to take a photo")

    if img_file:
        img = Image.open(img_file)
        img_path = "test.png"
        img.save(img_path)
        st.success("✅ Photo captured successfully!")

        decoded = decode_img(img_path)
        if decoded:
            go_to_result(decoded)
        else:
            st.error("⚠️ No QR code detected. Try again!")

# --- Page 2: Result Page ---
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})
    st.json(decoded)

    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"


In [ ]:
uploaded_file = st.file_uploader("Upload Image", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Create save folder if it doesn't exist
    save_dir = "uploaded_images"
    os.makedirs(save_dir, exist_ok=True)
    
    # Define save path
    save_path = os.path.join(save_dir, uploaded_file.name)
    
    # Save image automatically
    image = Image.open(uploaded_file)
    image.save(save_path)
    
    st.success(f"✅ Image saved successfully to: {save_path}")
else:
    st.info("👆 Upload an image to auto-save it.")

In [ ]:
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})

    if decoded:
        # --- Display the image in the center ---
        if "image" in decoded and decoded["image"]:
            st.markdown(
                f"""
                <div style="display: flex; justify-content: center; margin-top: 20px;">
                    <img src="{decoded['image']}" alt="User Image"
                        style="width: 250px; height: 250px; border-radius: 15px; object-fit: cover; box-shadow: 0 0 15px rgba(0,0,0,0.6);">
                </div>
                """,
                unsafe_allow_html=True
            )
            st.markdown("<br>", unsafe_allow_html=True)

        # --- Display the rest of the details below the image ---
        st.subheader("📄 Details")
        details_to_show = {k: v for k, v in decoded.items() if k != "image"}

        # Use a nice bordered container for details
        st.markdown(
            "<div style='background-color:#1a1b1e; padding: 20px; border-radius: 12px; box-shadow: 0 0 15px rgba(59,130,246,0.3);'>",
            unsafe_allow_html=True,
        )

        for key, value in details_to_show.items():
            st.markdown(f"<p style='font-size:18px;'><b>{key.title()}:</b> {value}</p>", unsafe_allow_html=True)

        st.markdown("</div>", unsafe_allow_html=True)

    else:
        st.warning("⚠️ No decoded data found!")

    # --- Button to scan another QR ---
    st.markdown("<br>", unsafe_allow_html=True)
    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"
        st.rerun()


In [ ]:
elif st.session_state.page == "result":
    st.title("✅ Decoded Information")

    decoded = st.session_state.get("decoded_data", {})
    st.json(decoded)

    if st.button("🔙 Scan Another"):
        st.session_state.page = "scanner"

In [ ]:
import streamlit as st

# ---- Initialize session state ----
if "page" not in st.session_state:
    st.session_state.page = "scanner"

# ---- SCANNER PAGE ----
if st.session_state.page == "scanner":
    st.title("📷 QR Scanner")
    st.write("Scan a QR or click below to view stored info.")

    # Example data (you can replace this with decoded QR data)
    st.session_state.decoded_data = {
        "Name": "Prashanth",
        "Blood Group": "B+",
        "Emergency Contact": "95991576744"
    }

    # 🔘 Button to navigate to Result page
    if st.button("👁️ View Info", use_container_width=True):
        st.session_state.page = "result"
        st.experimental_rerun()  # or st.rerun() in latest Streamlit

# ---- RESULT PAGE ----
elif st.session_state.page == "result":
    st.title("✅ Patient Information")

    decoded = st.session_state.get("decoded_data", {})
    if decoded:
        col1, col2, col3 = st.columns([1, 2, 1])
        with col2:
            # If you have a saved image path
            image_path = "test.png"
            if image_path:
                st.image(image_path, caption="🩺 Patient Photo", use_container_width=True)
            st.write("### 👇 Patient Details")
            st.json(decoded)
    else:
        st.warning("⚠️ No patient data found!")

    if st.button("🔙 Back to Scanner", use_container_width=True):
        st.session_state.page = "scanner"
        st.experimental_rerun()


In [ ]:
elif st.session_state.page == "scanner":
    st.title("📷 Scanner")

    # Run scan() only if data not already captured
    if "decoded_data" not in st.session_state:
        decoded = scan()
        if decoded is not None:
            st.session_state.decoded_data = decoded
            st.success("✅ QR decoded successfully!")
    else:
        st.info("✅ QR data already captured — click below to view info.")

    # Show the View Info button when decoded data exists
    if "decoded_data" in st.session_state:
        if st.button("View Info", use_container_width=True):
            st.session_state.page = "result"
            st.rerun()
